# 二分法
在有序数组中寻找target值
时间复杂度O(logN), 空间复杂度O(1)
Decrease and Conquer

Ex. 在有序数组中寻找3
下标  0  1  2  3  4  5  6
数值  1  2  3  4  7  8  9
(1)  l                 r
(2)           m
     l        r
(3)     m
        l     r
(4)        m: Found and return    
     
# 时间复杂度
复杂度	     可能对应的算法
O(logn)	    二分法	
O(n ^ 1/2)   分解质因数(很少)
O(n)	       枚举法，双指针算法，单调栈算法
O(nlogn)     排序, (N * logN) 的数据结构上操作
O(n^2, n^3)	 枚举法，动态规划
O(2^n)   	 与组合有关的搜索问题	
O(n!)	       与排列有关的搜索问题

1. 二分法模板
   - 不会死循环的二分法
   - 递归与非递归的权衡
2. 在排序的数据集上进行二分
   - 找到满足某个条件的第一个或者最后一个位置
3. 在未排序的数据集上进行二分
   - 保留有解的一半, 或去掉无解的一半
4. 在答案集上进行二分
   - 二分答案并验证答案偏大还是偏小 

In [ ]:
# [457] Binary Search
# 二分法模板(去comments)
def binarySearch(self, nums, target):
    if not nums:
        return -1

    start, end = 0, len(nums) - 1

    while start + 1 < end:  # 1. start + 1 < end 避免死循环, 退出条件 ptr_start 和 ptr_end 相邻
        mid = (start + end) // 2
        if nums[mid] < target:
            start = mid
        elif nums[mid] == target:
            end = mid # 3. 可以不 +/-1
        else: 
            end = mid

    # 2. 判断退出条件
    if nums[start] == target:
        return start
    if nums[end] == target:
        return end

    return -1

In [ ]:
# 死循环 
# ex. 寻找最后一个target, 等于target的最大index
# ex. [1, 1] 寻找最右1

# Ex. start > end  => start + 1 > end
# 在[1,1]中寻找最后一个1
# 下标  0  1  
# 数值  1  1 
# (1)  l  r
# (2)  m
#      l  r (ptr_left doesn't move after one iteration) 

# Ex. start = mid  => start = mid + 1
# 在[1,2]中寻找最后一个1
# 下标  0  1  
# 数值  1  2 
# (1)  l  r
# (2)  m
#         l/r (miss the target)
def lastPosition(self, nums, target):
    if not nums:
        return -1

    start, end = 0, len(nums) - 1

    while start < end:  # 1. start < end
        mid = (start + end) // 2
        if nums[mid] < target:
            start = mid  
        elif nums[mid] == target:
            start = mid # 3. (start = mid + 1) also doesn't work 
        else: 
            end = mid

    # 2. 判断退出条件
    if nums[end] == target:
        return end

    return -1

# 在排序的数据集上进行二分
给你一个数组, 找出数组中第一个/最后一个满足某个条件的位置 
OOOOOOO...OOXX....XXXXXX
           ^^      

In [ ]:
# [447] Search in a Big Sorted Array
# 这个数组很大只能通过固定的结构 ArrayReader.get(k)来访问第k个数
# 找到target第一次出现的位置

# hint: 算法时间复杂度 - O(logk), k为target第一次出现未知的下标

# Sol 1. Exponential Backoff 倍增法

# Ex. 寻找第一个6
# 下标  0
# 数值  1
#      ^ (nums[0] < 6)

# 下标  0  1
# 数值  1  3
#         ^ (nums[1] < 6)

# 下标  0  1  2  3
# 数值  1  3  3  5
#               ^ (nums[3] < 6)

# 下标  0  1  2  3  4  5  6  7
# 数值  1  3  3  5  6  6  6  7
#                   m        ^ (nums[7] > 6)

def search_big_sorted_array(self, reader, target):
    # 初始化查找范围为1, 代表在前1个数中查找
    range_total = 1
    # 倍增法: 如果target在查找范围之外,查找范围翻倍,时间复杂度O(logK)
    while reader.get(range_total - 1) < target:
        range_total = range_total * 2
    
    # start 也可以是 range_total // 2, 但是保守的写法是0,
    # 因为写成 0 也不会影响时间复杂度
    # 如果 target 在 range_total中, 则 index 的范围为 [0, range_total - 1]
    # 时间复杂度 O(logK)
    start, end = 0, range_total - 1
    
    while start + 1 < end:  # 1. start + 1 < end 避免死循环, 退出条件 ptr_start 和 ptr_end 相邻
        mid = (start + end) // 2
        if reader.get(mid) < target:
            start = mid
        # 要点: 为什么这里(target == 中间值), 不直接返回
        # 因为在中点左边还能存在更先出现的target值
        elif reader.get(mid) == target:
            end = mid        # 3. 可以不 +/-1
        else: 
            end = mid

    # 要点: 为什么这里先检查 start, 再检查 end ?
    # 因为我们要求的是 first index, 所以先检查更靠左的 start
    # 如果 start 不为 target, 再检查稍微靠右的 end 
    # 比如, 在 [9, 9] 中寻找9, 应返回0 (第一个9的 index)
    if reader.get(start) == target:
        return start
    if reader.get(end) == target:
        return end
    return -1

In [ ]:
# [460] Find K Closest Elements在排序数组中找最接近target的K个数

# 1.如果接近程度相当，那么小的数排在前面。

# Ex.  1   2 [3] 4  6  8
# 2.target有可能不存在于数组中.

# Sol:
# (1) 找到中界线(target的插入位置)
#                        不存在3               存在3
#
#     <=3 的最右数字  1  2 | 4  6  8      1   3   4   6   8 
#                      ^                     ^
#     >=3 的最左数字  1  2 | 4  6  8      1   3   4   6   8 
#                          ^                 ^
# (2) 从中接线两边找最近的 k 个数字 (Merge sorted Array)
#     Ex. target=3, k=4
#     下标  0   1     2   3   4                
#     数值  1   2  |  4   6   8 
#                  3
#              l     r           => 2 (same distance, 2 is smaller)
#          l         r           => 4
#          l             r       => 1
#       l                r       => 6
#       l                    r  

def k_closest_numbers(self, A, target, k):
    # 1. 找中界线: 二分法找到最左边 >= target 的元素
    right = self.find_upper_closest(A, target)
    left = right - 1

    # 2. 找最接近target的K个数: back2back pointers + merge sorted array
    results = []
    for _ in range(k):
        # 如果左边更接近, 选左边
        if self.is_left_closer(A, target, left, right):
            results.append(A[left])
            left -= 1
        else:
            results.append(A[right])
            right += 1
        return results
    
def find_upper_closest(self, A, target):
    """ 找到最左边的 >= target 的元素 """
    start, end = 0, len(A) - 1
    while start + 1 < end:
        mid = (start + end) // 2
        # 如果 mid >= target, mid 符合条件向左边去寻找
        # 更靠左的符合条件的元素, 丢掉右边
        if A[mid] >= target:
            end = mid
        # 如果 mid < target, >= target 的元素在右边, 丢掉左边
        else:
            start = mid
        
        # 因为需要找最左数,所以这里需要先判断start
        if A[start] >= target:
            return start
        if A[end] >= target:
            return end

        # 所有的值小于target
        return len(A)

def is_left_closer(self, A, target, left, right):
    # 如果左边已经耗尽, 返回 false
    if left < 0:
        return False
    
    # 如果右边已经耗尽, 返回 true
    if right >= len(A):
        return True

    # 为什么有等号? 如果左右距离相等, 选左边
    return target - A[left] <= A[right] - target


In [ ]:
# [585] Maximum Number in Mountain Seqence 山脉序列中的最大值
# 1.山脉数组(先增后减序列), 找到山顶(最大值)
# 2.没有重复点
#
#           8
#              6
#        4        3
#     2
#  1
# 
# Sol 1. 打擂台 O(n)
# Sol 2. 在排序数据集上进行二分 XXXOOO
#        peak在左分区判定条件: nums[mid] > nums[mid + 1]

def find_peak_of_mountain_array(self, nums):
    if not nums:
        return -1
    # 找到第一个符合条件 nums[i] > nums[i + 1] 的i
    start, end = 0, len(nums) - 1
    while start + 1 < end:
        mid = (start + end) // 2
        # mid + 1 一定不会越界
        # 因为 while 循环退出条件是 start + 1 < end,
        # 所以一定有 start < mid + 1 < end 
        # 如果从 mid 点向右下方倾斜, 山峰一定在左边, 丢掉右边
        if nums[mid] > nums[mid + 1]:
            end = mid
        # 如果从 mid 点向左下方倾斜, 山峰一定在右边, 丢掉左边
        else:
            start = mid
    # 返回 start 和 end 中较大的值, 则为山顶
    return max(nums[start], nums[end])

    # 如果输入数据只有两个数怎么办? 比如[1, 2] 
    # 为什么没有重复点是个很重要的条件? 平顶山就不知道向左还是向右了
    # 
    #      3                        |                        3
    #    2    2   2   2   2   2     |     2   2   2   2   2     2
    #  1                         1  |  1                           1
    #            mid                                 mid

In [ ]:
# [159] Find Minimum in Rotated Sorted Array 寻找旋转排序数组中的最小值
# Suppose a sorted array in ascending order is rotated at some pivot unknown to you beforehand. 
# (i.e., 0 1 2 4 5 6 7 might become 4 5 6 7 0 1 2).
# Find the minimum element.
# You can assume no duplicate exists in the array.
# 
# Example:
#       7
#     6
#   5
# 4
#              3
#            2
#         1*
# 
# min在左分区判定条件: mid > end
# 
def find_min(self, nums):
    # 特殊情况处理, 需要跟面试官确认特殊情况的返回值
    if not nums:
        return - 1
    
    start, end = 0, len(nums) - 1
    while start + 1 < end:
        mid = (start + end) // 2
        # 如果 mid > end, 起点在右边, 抛弃左边
        if nums[mid] > nums[end]:
            start = mid
        # 如果 mid < end, 起点在左边, 抛弃右边
        # Note: 题目声明不存在duplicate元素, 所以这里 else等价于 mid < end
        else:
            end = mid
    # 返回 start 和 end 的最小值
    return max(nums[start], nums[end])


In [ ]:
# [62] Search for Target k in Rotated Sorted Array 搜索旋转排序数组
# Suppose a sorted array is rotated at some pivot unknown to you beforehand.
# (i.e., 0 1 2 4 5 6 7 might become 4 5 6 7 0 1 2).
# You are given a target value to search. 
# If found in the array return its index, otherwise return -1. 
# 
# Constraints: You may assume no duplicate exists in the array.
# 
# Example:
#       7
#     6
#   5
# 4
#             3
#           2
#         1
# 
# Sol1. 用两次二分法即可
#      (1) 第一次二分找到最小值
#      (2) 第二次二分在左上或右下区间找目标值
# Sol2. 能否只用一次二分就解决这个问题?


# 在有序的输入集中的二分查找

本质: 在有序的数组中寻找一个跟target有关的index或值
时间复杂度: O(logN)
关键词:
array, target, sorted, equal or close to target(e.g.,小于n的最大值),复杂度 O(NlogN)

# 在未排序的数据集上进行二分
无法找到一个条件,形成xxoo的模型
但可以根据判断,保留下有解得那一半或者去掉无解的一半

In [ ]:
# [75] Find Peak Element

# Description: 
# There is an integer array which has the following features: The numbers in adjacent positions are different.
# A[0] < A[1] && A[A.length - 2] > A[A.length - 1].
# We define a position P is a peak if:
# A[P] > A[P-1] && A[P] > A[P+1] Find a peak element in this array. Return the index of the peak.

# Constraints:
# It's guaranteed the array has at least one peak.
# The array may contain multiple peeks, find any of them. 
# The array has at least 3 numbers in it.

# Example:
# Input:  [1, 2, 1, 3, 4, 5, 7, 6] 
# Output:  1 or 6 (Output Index)
#
# Index: 0 1  2 3 4 5 6  7
# Value:              7* 
#                   5    6
#                 4
#               3
#          2*   
#        1    1

# 已知条件: A[0] < A[1] 并且 A[n - 2] > A[n - 1] 翻译成人话(画图)，就是两端都是向中间上升的趋势(或者理解为两端先升后降)
# (1) 两端都向中间下降，可能没有山峰
#  
#    2     2*     2    |    2           2
#       1      1       |        1   1  
#
# (2) 只有一端向中间上升，可能没有山峰
#             4    |            4    |  4            |  4
#       3*         |         3       |        3*     |     3
#          2       |      2          |     2         |        2
#    1             |   1             |           1   |           1
# 
# (3) 两端都是向中间上升的趋势，一定有山峰!!!
#                  |                 |    平顶山(不可能,已知相邻位置数值不同 A[P] > A[P-1] && A[P] > A[P+1])
#          3*      |           3*    | 
#       2          |      2*         |      2   2
#    1        1    |   1     1    1  |   1         1
# 
# 分情况讨论:
#                 |               |               | 
#                 |   3           |           3   | 
#    2       2    |       2       |       2       |       2
#        1        |           1   |   1           |   1       1
#       mid       |      mid      |       mid     |      mid
#      左/右               左               右            Found
# peak 在左边的判定条件: mid 是在下降序列上
# 

def find_peak(self, A):
# Peak 不可能在两端,所以在[1, A.length - 2]范围内寻找
    start, end = 1, len(A) - 2
    while start + 1 < end:
        mid = (start + end) // 2
        # 如果 mid 向左上方倾斜, 选左半边
        if A[mid] < A[mid - 1]:
            end = mid
        # 如果 mid 向右上方倾斜, 选右半边   
        elif A[mid] < A[mid + 1]:
            start = mid
        else:
            return mid
    # 因为保证一定有 peak, 所以返回 start 和 end 中大的一个
    return end if A[start] < A[end] else start